<img width=150 src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/NumPy_logo.svg/200px-NumPy_logo.svg.png"></img>

# NumPy 陣列進階操作

In [1]:
import numpy as np

---
## 1. NumPy 陣列重塑

### 1.1 `flatten()` 與 `ravel()`

透過 `flatten()` 與 `ravel()` 均可將多維陣列轉形為一維陣列，`flatten()` 與 `ravel()` 的使用透過下列兩種方法，得到的結果都是完全一樣的。

|np.函式|陣列物件.函式|
|---|---|
|np.flatten(a, order='C')|a.flatten(order='C')|
|np.ravel(a, order='C')|a.ravel(order='C')|

In [2]:
a = np.array([[ 0,  1,  2,  3], 
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [3]:
a.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

與 `flatten()` 不同的是，`ravel()` 建立的是原來陣列的 view<br>
所以在 `ravel()` 回傳物件中做的元素值變更，將會影響原陣列的元素值。

In [4]:
b = a.ravel()
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

如果我們改變 b 陣列的元素值，原陣列 a 對應的元素值也會被改變。

In [10]:
b[3] = 100
b

array([  0,   1,   2, 100,   4,   5,   6,   7,   8,   9,  10,  11])

In [11]:
a

array([[  0,   1,   2, 100],
       [  4,   5,   6,   7],
       [  8,   9,  10,  11]])

`flatten()` 與 `ravel()` 引數 order 預設值為 C，常用的引數值有 C 和 F。
* C 的意義是 C-style，展開時是以 row 為主的順序展開
* F 是 Fortran-style，展開時是以 column 為主的順序展開。

In [12]:
a.ravel(order='C')

array([  0,   1,   2, 100,   4,   5,   6,   7,   8,   9,  10,  11])

In [13]:
a.ravel(order='F')

array([  0,   4,   8,   1,   5,   9,   2,   6,  10, 100,   7,  11])

### 1.2 `reshape()`

`reshape()` 的使用，可以透過 `np.reshape(a, new_shape)` 或 `a.reshape(new_shape, refcheck=True)` 來執行。

In [15]:
a = np.arange(15)
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [11]:
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [16]:
a.reshape((3, 5))

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

如果新的總數與原先 shape 總數不一致的話，則會產生錯誤。

In [17]:
a.size

15

In [18]:
a.reshape((3, 6))

ValueError: cannot reshape array of size 15 into shape (3,6)

Reshape 時，新的形狀可以採用模糊指定為 -1，讓 NumPy 自動計算。

In [19]:
a.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

若 reshape 後的陣列元素值改變，會影響原陣列對應的元素值也跟著改變。

In [20]:
b = a.reshape((3, 5))
b[0, 2] = 100
b

array([[  0,   1, 100,   3,   4],
       [  5,   6,   7,   8,   9],
       [ 10,  11,  12,  13,  14]])

a[2] 值被改變了。

In [21]:
a

array([  0,   1, 100,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14])

---
### 1.3 `resize()`

`resize()` 的使用，也同樣可以透過 `np.resize(a, new_shape)` 或 `a.resize(new_shape, refcheck=True)` 來執行。

In [26]:
b = np.arange(15)
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

如果 resize 的大小超過總元素值，則會在後面的元素值的指定為 0。

In [23]:
b.size

15

In [24]:
b.resize((3, 6), refcheck=False)
b

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14,  0,  0,  0]])

要改變被 reference 的陣列時有可能會產生錯誤，這時候可以將 `refcheck` 引數設為 `False` (預設為 `True`)，否則會報錯。

In [27]:
b.resize((3, 6), refcheck=True)
b

ValueError: cannot resize an array that references or is referenced
by another array in this way.
Use the np.resize function or refcheck=False

如果 resize 的大小小於總元素值，則會依照 C-style 的順序，取得 resize 後的陣列元素。

In [28]:
b.resize(3, refcheck=False)
b

array([0, 1, 2])

---
## 2. 軸 (axis) 與維度 (dimension)

軸 (axis) 在 NumPy 多維陣列中是很重要觀念，但是在應用上容易混淆。軸的數目也就是 NumPy 陣列的維度 (dimension) 數，軸的順序編號從 0 開始，下面例子用圖示來解說。

### 2.1 一維陣列的軸

對一維陣列來說，只有一個軸，所以 axis 為 0。

In [29]:
a = np.array([1, 2, 3])
a

array([1, 2, 3])

以圖示來說明一維陣列的軸。

![](1D_axis.png)

### 2.2 二維陣列的軸

二維陣列的 ndim 為 2，也就是會有 2 個軸。二維陣列的軸 0 就是沿著 row 的軸，而軸 1 是沿著 column 的軸。

In [30]:
a = np.arange(6).reshape(3, 2)
a

array([[0, 1],
       [2, 3],
       [4, 5]])

以圖示來說明二維陣列的軸。

![](2D_axis.png)

### 2.3 三維陣列的軸

三維陣列有 3 個軸。可以理解軸的順序是"由外而內"、**"由 row 而 column"**。

以前一天範例程式中三維陣列的例子來看，可以理解為 2 個 4 $\times$ 3 的二維陣列排在一起。

In [31]:
a = np.array([[[1, 2, 3], [4, 5, 6],
              [7, 8, 9], [10, 11, 12]],
              [[1, 2, 3], [4, 5, 6],
              [7, 8, 9], [10, 11, 12]]])
a

array([[[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]],

       [[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]]])

以圖示來說明三維陣列的軸。

![](3D_axis.png)

從 `shape` 屬性來看也可以協助理解在多維陣列中的軸。

In [34]:
a.shape  # (axis_0, axis_1, axis_2)

(2, 4, 3)

若我們要沿軸對元素做加總，呼叫 `sum()` 函式並指定 axis。

In [35]:
a.sum(axis=0)

array([[ 2,  4,  6],
       [ 8, 10, 12],
       [14, 16, 18],
       [20, 22, 24]])

### 2.4 `np.newaxis` 增加軸數

跟 `reshape()` 類似的應用，如果是要增加軸數的話，可以使用 `np.newaxis` 物件。將 `np.newaxis` 加到要增加的軸的位置即可。

與 `reshape()` 不同的地方在於，`np.newaxis` 新增的維度為 1，而 `reshape()` 可以指定要改變的形狀 (不一定為 1)。

In [48]:
a = np.arange(12).reshape(2, 6)
a

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11]])

In [49]:
a.ndim

2

In [50]:
a[:,np.newaxis,:].shape

(2, 1, 6)

In [51]:
a[:,np.newaxis,:].ndim

3

---
## 3. NumPy 陣列的合併與分割

In [54]:
a = np.arange(10).reshape(5, 2)
b = np.arange(6).reshape(3, 2)
a, b

(array([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]]),
 array([[0, 1],
        [2, 3],
        [4, 5]]))

### 3.1 合併：`concatenate()`, `stack()`, `hstack()`, `vstack()`

使用 `concatenate()` 進行陣列的合併時，須留意除了指定的軸之外 (預設為 axis 0)，其他軸的形狀必須完全相同，合併才不會發生錯誤。

```python
numpy.concatenate((a1, a2, ...), axis=0, out=None)
```

In [55]:
np.concatenate((a, b))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9],
       [0, 1],
       [2, 3],
       [4, 5]])

In [60]:
np.concatenate((a, b), axis=1)   # ValueError

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 5 and the array at index 1 has size 3

`stack()`, `hstack()`, `vstack()` 的觀念及用法類似，不同點在於 `stack()` 回傳的陣列維度會是合併前的維度 +1，而 `hstack()` 與 `vstack()` 回傳的陣列維度則是依合併的陣列而定。

至於是否可以合併，`stack()` 必須要所有陣列的形狀都一樣；而 `hstack()` 與 `vstack()` 則跟上述的規則一樣，除了指定的軸之外，其他軸的形狀必須完全相同才可以合併。

|函式|說明|
|---|---|
|numpy.stack(arrays, axis=0, out=None)|根據指定的軸進行合併|
|numpy.hstack(tup)|根據水平軸進行合併|
|numpy.vstack(tup)|根據垂直軸進行合併|

* `stack()` 範例

In [62]:
c = np.arange(10).reshape(5, 2)
a, c

(array([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]]),
 array([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]]))

In [66]:
stack = np.stack((a, c), axis=1)
stack

array([[[0, 1],
        [0, 1]],

       [[2, 3],
        [2, 3]],

       [[4, 5],
        [4, 5]],

       [[6, 7],
        [6, 7]],

       [[8, 9],
        [8, 9]]])

In [68]:
a.shape, c.shape, stack.shape

((5, 2), (5, 2), (5, 2, 2))

* `hstack()` 範例

In [69]:
np.hstack((a, c))

array([[0, 1, 0, 1],
       [2, 3, 2, 3],
       [4, 5, 4, 5],
       [6, 7, 6, 7],
       [8, 9, 8, 9]])

* `vstack()` 範例

In [76]:
np.vstack((a, b))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9],
       [0, 1],
       [2, 3],
       [4, 5]])

In [75]:
np.vstack((a, b)) == np.concatenate((a, b))

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]])

### 3.2 分割：`split()`、`hsplit()`、`vsplit()`

In [77]:
a = np.arange(10).reshape(5, 2)
a

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

`split()` 的語法：

```python
numpy.split(array, indices_or_sections, axis=0)
```

param `indices_or_sections`
* 如果給定單一整數的話，那就會按照軸把陣列等分
* 如果給定一個 List 的整數值的話，就會按照區段去分割
* 例如：`indices_or_sections=[2, 3]` 會照下列的方式做分割 (一樣是按照軸把陣列分割)
```
arr[:2]
arr[2:3]
arr[3:]
```

與 `split` 很類似的是 `hsplit` 與 `vsplit`，分別是依照水平軸和垂直軸去做分割。

* `split()`

In [81]:
# 依 axis 0 等分 split
np.split(a, 5)

# 若無法等分，會 ValueError: array split does not result in an equal division

[array([[0, 1]]),
 array([[2, 3]]),
 array([[4, 5]]),
 array([[6, 7]]),
 array([[8, 9]])]

In [82]:
# split 為 (2,2), (1,2), (2,2) 三個陣列，並回傳含 3 個陣列的 List
np.split(a, [2,3])

[array([[0, 1],
        [2, 3]]),
 array([[4, 5]]),
 array([[6, 7],
        [8, 9]])]

* `hsplit()`

In [83]:
b = np.arange(30).reshape(5, 6)
b

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29]])

In [84]:
# 依水平軸去做等分分割
np.hsplit(b, 3)

[array([[ 0,  1],
        [ 6,  7],
        [12, 13],
        [18, 19],
        [24, 25]]),
 array([[ 2,  3],
        [ 8,  9],
        [14, 15],
        [20, 21],
        [26, 27]]),
 array([[ 4,  5],
        [10, 11],
        [16, 17],
        [22, 23],
        [28, 29]])]

In [86]:
# 依水平軸照區段去分割
np.hsplit(b, [2, 3, 5])  # [:2] | [2:3] | [3:5] | [5:] 

[array([[ 0,  1],
        [ 6,  7],
        [12, 13],
        [18, 19],
        [24, 25]]),
 array([[ 2],
        [ 8],
        [14],
        [20],
        [26]]),
 array([[ 3,  4],
        [ 9, 10],
        [15, 16],
        [21, 22],
        [27, 28]]),
 array([[ 5],
        [11],
        [17],
        [23],
        [29]])]

* `vsplit()`

In [87]:
# 依垂直軸按照區段去分割，超出的區段則傳回空陣列
np.vsplit(b, [2, 4, 6])

[array([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]]),
 array([[12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]),
 array([[24, 25, 26, 27, 28, 29]]),
 array([], shape=(0, 6), dtype=int64)]

---
## 4. 迭代

一維陣列的迭代，跟 Python 集合型別 (例如 List) 的迭代相同。

In [88]:
a = np.arange(5)
a

array([0, 1, 2, 3, 4])

In [89]:
for i in a:
    print(i)

0
1
2
3
4


多維陣列的迭代則以 axis 0 為準。下面以二維陣列為例，列出各 row 的元素。

In [90]:
b = np.arange(6).reshape(2, 3)
b

array([[0, 1, 2],
       [3, 4, 5]])

In [91]:
for row in b:
    print(row)

[0 1 2]
[3 4 5]


如果要列出多維陣列所有元素的話，可以配合 `flat` 屬性。

In [92]:
for i in b.flat:
    print(i)

0
1
2
3
4
5


---
## 5. 搜尋與排序

### 5.1 顯示最大值和最小值：`amax()`、`amin()`、`max()`、`min()`

要顯示陣列元素最大值和最小值，可以透過 `amax()`、`amin()`、`max()`、`min()`，也可以依照軸列出各軸的最大/最小元素值。

基本語法：

|np.函式|陣列物件.函式|
|---|---|
|numpy.amax(array, axis=None, keepdims=<no value>)|ndarray.max(axis=None, keepdims=False)|
|numpy.amin(array, axis=None, keepdims=<no value>)|ndarray.min(axis=None, keepdims=False)|

In [93]:
a = np.random.randint(1, 20, 10)
a

array([ 6,  1,  4,  3,  9,  9,  8,  8,  7, 19])

In [94]:
# 陣列中最大的元素值
np.amax(a)

19

In [95]:
# 陣列中最小的元素值
np.amin(a)

1

如果是多維陣列的話，用法也是相同，也可以依照軸列出最大或最小值。

In [96]:
b = a.reshape(2, 5)
b

array([[ 6,  1,  4,  3,  9],
       [ 9,  8,  8,  7, 19]])

In [97]:
# 若設定 keepdims=True，結果會保留原陣列的維度來顯示。
np.amax(b, keepdims=True)

array([[19]])

In [98]:
# 列出各 row 最大值
b.max(axis=1)

array([ 9, 19])

In [99]:
# 同樣的 amax 也可以依軸列出各 row 最大值
np.amax(b, axis=1)

array([ 9, 19])

In [100]:
# 列出各 column 最小值
b.min(axis=0)

array([6, 1, 4, 3, 9])

### 5.2 顯示最大值和最小值的索引：`argmax()` 與 `argmin()`

`argmax` / `argmin` 和上述不同的地方在於，`argmax` / `argmin` 回傳的是最大值和最小值的索引，也可以依照軸找出各軸最大值和最小值的索引。

基本語法：

|np.函式|陣列物件.函式|
|---|---|
|numpy.argmax(array, axis=None)|ndarray.argmax(axis=None)|
|numpy.argmin(array, axis=None)|ndarray.argmin(axis=None)|

In [101]:
np.random.seed(0)
a = np.random.randint(1, 20, size=(3, 4))
a

array([[13, 16,  1,  4],
       [ 4,  8, 10, 19],
       [ 5,  7, 13,  2]])

若沒有指定軸的話，`argmax()` 與 `argmin()` 會回傳多維陣列展平後的索引。

In [102]:
np.argmax(a)  # a.max() == a.flat[7] = [19] 

7

In [103]:
# 列出各 column 的最大值索引, 分別為 [0, 0, 2, 1]
np.argmax(a, axis=0)

array([0, 0, 2, 1])

In [59]:
# 元素值 1 為最小值，展平後的索引值為 2。
a.argmin()

2

### 5.3 找出符合條件的元素：`where`

語法：
```python
numpy.where(condition[, x, y])
```

In [109]:
a

array([[13, 16,  1,  4],
       [ 4,  8, 10, 19],
       [ 5,  7, 13,  2]])

傳入條件式，回傳值為符合條件的元素索引，不過這邊留意的是，以下面二維陣列為例，回傳的索引陣列要合併一起看，也就是說
```
(array([0, 0, 1, 2]), 
 array([0, 1, 3, 2]))
```

a[0, 0] 值為 13<br />
a[0, 1] 值為 16<br />
a[1, 3] 值為 19<br />
a[2, 2] 值為 13

以上索引值對應的元素，其值都符合 "大於 10" 的條件。

In [110]:
np.where(a > 10)

(array([0, 0, 1, 2]), array([0, 1, 3, 2]))

若是設定 x, y 引數的話，可將各元素取代掉。<br>以下面的例子來解釋，如果元素值大於 10 的話就用 "Y" 來替代，反之則是 "N"。

In [111]:
np.where(a > 10, "Y", "N")

array([['Y', 'Y', 'N', 'N'],
       ['N', 'N', 'N', 'Y'],
       ['N', 'N', 'Y', 'N']], dtype='<U1')

### 5.4 `nonzero`

`nonzero` 等同於 `np.where(array != 0)` 的語法，同樣的也是回傳符合非 0 條件的元素索引值。

語法：

|np.函式|陣列物件.函式|
|---|---|
|numpy.nonzero(array)|ndarray.nonzero()|

In [113]:
np.random.seed(2)
a = np.random.randint(0, 5, 10)
a

array([0, 0, 3, 2, 3, 0, 2, 1, 3, 2])

In [114]:
np.nonzero(a)

(array([2, 3, 4, 6, 7, 8, 9]),)

In [115]:
a.nonzero()

(array([2, 3, 4, 6, 7, 8, 9]),)

### 5.5 排序：`sort()` 與 `argsort()`

要對陣列進行排序可以使用 `sort()` 與 `argsort()`，兩者的差異是在 `sort()` 回傳的是排序後的陣列，而 `argsort()` 回傳的是排序後的陣列索引值。

語法：

|np.函式|陣列物件.函式|
|---|---|
|numpy.sort(a, axis=-1, kind=None, order=None)|ndarray.sort()|
|numpy.argsort(a, axis=-1, kind=None, order=None)|ndarray.argsort()|

In [120]:
np.random.seed(3)
a = np.random.randint(0, 20, 10)
a

array([10,  3,  8,  0, 19, 10, 11,  9, 10,  6])

In [121]:
np.sort(a)

array([ 0,  3,  6,  8,  9, 10, 10, 10, 11, 19])

In [122]:
a.argsort()

array([3, 1, 9, 2, 7, 0, 5, 8, 6, 4])

與 `np.sort()` 不同的是，`陣列物件.sort()` 的語法會進行 in-place 排序，也就是原本的陣列內容會跟著改變。

In [123]:
a

array([10,  3,  8,  0, 19, 10, 11,  9, 10,  6])

In [124]:
a.sort()
a

array([ 0,  3,  6,  8,  9, 10, 10, 10, 11, 19])

多維陣列在排序時可以指定要依據的軸。

In [125]:
b = np.random.randint(0, 20, size=(5, 4))
b

array([[ 0, 12,  7, 14],
       [17,  2,  2,  1],
       [19,  5,  8, 14],
       [ 1, 10,  7, 11],
       [ 1, 15, 16,  5]])

In [126]:
np.sort(b, axis=0)

array([[ 0,  2,  2,  1],
       [ 1,  5,  7,  5],
       [ 1, 10,  7, 11],
       [17, 12,  8, 14],
       [19, 15, 16, 14]])

In [127]:
np.sort(b, axis=1)

array([[ 0,  7, 12, 14],
       [ 1,  2,  2, 17],
       [ 5,  8, 14, 19],
       [ 1,  7, 10, 11],
       [ 1,  5, 15, 16]])

排序支援多種不同的排序算法，包括 quicksort (預設)、heapsort、mergesort、timsort，在 `kind` 引數指定即可。依照官網文件指出排序速度是以 quicksort 最快，mergesort / timsort 其次，之後是 heapsort。

In [162]:
c = np.random.randint(0, 100000000, 1000000)
c

array([41596254, 85298542,  4022792, ...,  3230884, 23288265, 33304365])

In [163]:
%%time 
np.sort(c)

CPU times: user 74.9 ms, sys: 3.2 ms, total: 78.1 ms
Wall time: 77.3 ms


array([       6,      115,      253, ..., 99999645, 99999779, 99999930])

In [164]:
%%time
np.sort(c, kind='heapsort')

CPU times: user 153 ms, sys: 4.55 ms, total: 157 ms
Wall time: 158 ms


array([       6,      115,      253, ..., 99999645, 99999779, 99999930])

In [165]:
%%time 
np.sort(c, kind='mergesort')

CPU times: user 65.7 ms, sys: 3.52 ms, total: 69.3 ms
Wall time: 68.6 ms


array([       6,      115,      253, ..., 99999645, 99999779, 99999930])

In [170]:
%%time
np.sort(c, kind='stable')

CPU times: user 66.1 ms, sys: 3.28 ms, total: 69.4 ms
Wall time: 68.7 ms


array([       6,      115,      253, ..., 99999645, 99999779, 99999930])

`kind：{‘quicksort’, ‘mergesort’, ‘heapsort’, ‘stable’}, optional`<br>
* Note that both ‘stable’ and ‘mergesort’ use timsort or radix sort under the covers and, in general, the actual implementation will vary with data type. (timsort 已被改成在 mergesort 和 stable 使用)
* 這個例子是 timsort 系列最快，quicksort 第二，heapsort 最慢

In [171]:
%time np.sort(c, kind='timsort')

ValueError: timsort is an unrecognized kind of sort